In [181]:
#-----IMPORTANDO ARQUIVO CSV E CRIANDO DATA FRAME 'df_ocorrencia'-----#

import os 
import pandas as pd
import numpy as np 
import psycopg2 as pg

df_ocorrencia = pd.read_csv('C:\\Users\\josen\\Desktop\\Projetos\\ocorrencia.csv', encoding='utf-8', delimiter=';')

In [182]:
#-----CORREÇÃO DE ERROS NA FORMATAÇÃO DOS DADOS-----#

import warnings


#-----Dados com mais de um '-'
num_celulax_lon = 0

while num_celulax_lon <= 6658:
    dado_com_erro_lon = df_ocorrencia['ocorrencia_longitude'].iloc[num_celulax_lon]
    dado_com_erro_lon = str(dado_com_erro_lon)
    numSinal = dado_com_erro_lon.count('-')
    
    if numSinal >= 2:
        str_para_rep = dado_com_erro_lon[0]
        df_ocorrencia['ocorrencia_longitude'].iloc[num_celulax_lon] = df_ocorrencia['ocorrencia_longitude'].iloc[num_celulax_lon].replace(str_para_rep, '')
    
    num_celulax_lon = num_celulax_lon + 1
    
    
#-----Células com  dados inválidos
df_ocorrencia = df_ocorrencia.replace('***', '')
df_ocorrencia['ocorrencia_latitude'] = df_ocorrencia['ocorrencia_latitude'].replace(u"DESCONHECIDA", "", regex=True)
df_ocorrencia['ocorrencia_longitude'] = df_ocorrencia['ocorrencia_longitude'].replace(u"DESCONHECIDA", "", regex=True)
df_ocorrencia['ocorrencia_latitude'] = df_ocorrencia['ocorrencia_latitude'].replace("***", "")
df_ocorrencia['ocorrencia_longitude'] = df_ocorrencia['ocorrencia_longitude'].replace("***", "")
df_ocorrencia['ocorrencia_latitude'] = df_ocorrencia['ocorrencia_latitude'].replace(u"NÃOINFORMADO", "", regex=True)
df_ocorrencia['ocorrencia_longitude'] = df_ocorrencia['ocorrencia_longitude'].replace(u"NÃOINFORMADO", "", regex=True)


#-----Substituindo ',' por '.' para que os dados sejam tratados como numéricos
df_ocorrencia['ocorrencia_latitude'] = df_ocorrencia['ocorrencia_latitude'].replace(u",", ".", regex=True)
df_ocorrencia['ocorrencia_longitude'] = df_ocorrencia['ocorrencia_longitude'].replace(u",", ".", regex=True)


#-----Pequenos erros de formatação
df_ocorrencia['ocorrencia_latitude'] = df_ocorrencia['ocorrencia_latitude'].replace(u"°", "", regex=True)
df_ocorrencia['ocorrencia_longitude'] = df_ocorrencia['ocorrencia_longitude'].replace(u"°", "", regex=True)
df_ocorrencia['ocorrencia_latitude'] = df_ocorrencia['ocorrencia_latitude'].replace(u" ", "", regex=True)
df_ocorrencia['ocorrencia_longitude'] = df_ocorrencia['ocorrencia_longitude'].replace(u" ", "", regex=True)
df_ocorrencia['ocorrencia_latitude'] = df_ocorrencia['ocorrencia_latitude'].replace(u"W", "", regex=True)
df_ocorrencia['ocorrencia_longitude'] = df_ocorrencia['ocorrencia_longitude'].replace(u"W", "", regex=True)
df_ocorrencia['ocorrencia_latitude'] = df_ocorrencia['ocorrencia_latitude'].replace(u"S", "", regex=True)
df_ocorrencia['ocorrencia_longitude'] = df_ocorrencia['ocorrencia_longitude'].replace(u"S", "", regex=True)
df_ocorrencia['ocorrencia_latitude'] = df_ocorrencia['ocorrencia_latitude'].str.replace("\\","")
df_ocorrencia['ocorrencia_longitude'] = df_ocorrencia['ocorrencia_longitude'].str.replace("\\","")
df_ocorrencia['ocorrencia_latitude'] = df_ocorrencia['ocorrencia_latitude'].str.replace("*","")
df_ocorrencia['ocorrencia_longitude'] = df_ocorrencia['ocorrencia_longitude'].str.replace("*","")
df_ocorrencia['ocorrencia_longitude'] = df_ocorrencia['ocorrencia_longitude'].str.replace("_","")
df_ocorrencia['ocorrencia_latitude'] = df_ocorrencia['ocorrencia_latitude'].str.replace("_","")
df_ocorrencia['ocorrencia_longitude'] = df_ocorrencia['ocorrencia_longitude'].str.replace("_","")
df_ocorrencia['ocorrencia_latitude'] = df_ocorrencia['ocorrencia_latitude'].str.replace("’","")
df_ocorrencia['ocorrencia_longitude'] = df_ocorrencia['ocorrencia_longitude'].str.replace("’","")
df_ocorrencia['ocorrencia_latitude'] = df_ocorrencia['ocorrencia_latitude'].str.replace("”","")
df_ocorrencia['ocorrencia_longitude'] = df_ocorrencia['ocorrencia_longitude'].str.replace("”","")                                                      
df_ocorrencia['ocorrencia_latitude'] = df_ocorrencia['ocorrencia_latitude'].str.replace(u't', '')
df_ocorrencia['ocorrencia_longitude'] = df_ocorrencia['ocorrencia_longitude'].str.replace(u't','')

warnings.filterwarnings('ignore')

In [ ]:
#------REPLACE EM DADOS PREENCHIDOS COMO 'Longitude:-xxxxx'-----#
num_celula_lon = 0
while num_celula_lon < 6658:
    
    linha_com_erro = df_ocorrencia['ocorrencia_longitude'].iloc[num_celula_lon]
    linha_com_erro = str(linha_com_erro)
    
    if linha_com_erro != '':
        if linha_com_erro[0] == 'L':
            df_ocorrencia['ocorrencia_longitude'].iloc[num_celula_lon] = df_ocorrencia['ocorrencia_longitude'].iloc[num_celula_lon].replace(linha_com_erro, '')
            
    num_celula_lon = num_celula_lon + 1
    

#------REPLACE EM DADOS PREENCHIDOS COMO 'Latitude:-xx'-----#
num_celula_lat = 0
while num_celula_lat < 6658:
    
    linha_com_erro_lat = df_ocorrencia['ocorrencia_latitude'].iloc[num_celula_lat]
    linha_com_erro_lat = str(linha_com_erro_lat)
    
    if linha_com_erro_lat != '':
        if linha_com_erro_lat[0] == 'L':
            df_ocorrencia['ocorrencia_latgitude'].iloc[num_celula_lat] = df_ocorrencia['ocorrencia_latitude'].iloc[num_celula_lat].replace(linha_com_erro_lat, '')
            
    num_celula_lat = num_celula_lat + 1

Alguns dados de latitude e longitude estavam com erros de formatação que implicaram na adição de '.' (pontos) a cada
3 caracteres, o que impossibilita seu uso e/ou recuperação. Porém outros dados não apresentavam o mesmo problema.
A solução a seguir foi pensada a fim de aproveitar a maior quantidade de dados viáveis possíveis e retirar da tabela 
somente as células com mais de um '.', substituindo as mesmas por células vazias.

In [183]:
#-----LISTA COM CÉLULAS QUE POSSUEM MAIS DE UM PONTO EM LONGITUDE:
celula_x = 0
listaValores_replace_lon = []

while celula_x < 6658:
    linha_lon = df_ocorrencia['ocorrencia_longitude'].iloc[celula_x]
    linha_lon = str(linha_lon)
    valorRep_lon = f'{linha_lon}'
    num_pontos_lon = (linha_lon.count("."))
    
    if num_pontos_lon >= 2:
        listaValores_replace_lon.append(valorRep_lon)
        
    celula_x = celula_x + 1    

     
#-----LISTA COM CÉLULAS QUE POSSUEM MAIS DE UM PONTO EM LATITUDE:
celula_y = 0
listaValores_replace_lat = []

while celula_y < 6658:
    linha_lat = df_ocorrencia['ocorrencia_latitude'].iloc[celula_y]
    linha_lat = str(linha_lat)
    valorRep_lat = f'{linha_lat}'
    num_pontos_lat = (linha_lat.count("."))
    
    if num_pontos_lat >= 2:
        listaValores_replace_lat.append(valorRep_lat)  
        
    celula_y = celula_y + 1

    
#-----REPLACE LONGITUDE COM BASE NA LISTA ANTERIORMENTE CRIADA:
elemento_x = 0
numElementos_rep_lon = len(listaValores_replace_lon)

while elemento_x < numElementos_rep_lon:
    valor_rep_lon = listaValores_replace_lon[elemento_x]
    df_ocorrencia['ocorrencia_longitude'] = df_ocorrencia['ocorrencia_longitude'].replace(valor_rep_lon, '', regex=True)
    elemento_x = elemento_x + 1
 

#-----REPLACE LATITUDE COM BASE NA LISTA ANTERIORMENTE CRIADA:
elemento_y = 0
numElementos_rep_lat = len(listaValores_replace_lat)

while elemento_y < numElementos_rep_lat:
    valor_rep_lat = listaValores_replace_lat[elemento_y]
    df_ocorrencia['ocorrencia_latitude'] = df_ocorrencia['ocorrencia_latitude'].replace(valor_rep_lat, '', regex=True)
    elemento_y = elemento_y + 1

In [189]:
#-----CONECTANDO AO BANCO DE DADOS-----#

host = 'localhost'
dbname = 'projeto_acidentes_aeroviarios'
user = '********'
password = '*******'
sslmode = 'disable'

conect_string = 'host={} user={} dbname={} password={} sslmode={}'.format(host, user, dbname, password, sslmode)
conec = pg.connect(conect_string)
cursor = conec.cursor()

print('Conectado ao banco de dados {} com sucesso'.format(dbname))

Conectado ao banco de dados projeto_acidentes_aeroviarios com sucesso


In [157]:
#-----GERANDO NOMES DAS COLUNAS PARA AUTOMATIZAR O COMANDO DE CRIAÇÃO DE TABELA-----#

tipos_de_dados_formatpostgr = {
    'object' : 'varchar',
    'int32' : 'integer',
    'int64' : 'integer',
    'datetime64[ns]' : 'date',
    'float64' : 'numeric'
}
colunas_formatadas = ', '.join('{} {}'.format(n, d) for (n, d) in zip(df_ocorrencia.columns, df_ocorrencia.dtypes.replace(tipos_de_dados_formatpostgr)))
colunas_ocorrencia = colunas_formatadas.replace('horario varchar', 'horario time with time zone')
colunas_ocorrencia

'codigo_ocorrencia integer, codigo_ocorrencia1 integer, codigo_ocorrencia2 integer, codigo_ocorrencia3 integer, codigo_ocorrencia4 integer, ocorrencia_classificacao varchar, ocorrencia_latitude varchar, ocorrencia_longitude varchar, ocorrencia_cidade varchar, ocorrencia_uf varchar, ocorrencia_pais varchar, ocorrencia_aerodromo varchar, ocorrencia_dia varchar, ocorrencia_hora varchar, investigacao_aeronave_liberada varchar, investigacao_status varchar, divulgacao_relatorio_numero varchar, divulgacao_relatorio_publicado varchar, divulgacao_dia_publicacao varchar, total_recomendacoes integer, total_aeronaves_envolvidas integer, ocorrencia_saida_pista varchar'

In [187]:
#-----CRIANDO NOVA TABELA DENTRO DO BANCO DE DADOS-----#

nome_da_tabela = 'ocorrencia'
colunas = 'codigo_ocorrencia integer, codigo_ocorrencia1 integer, codigo_ocorrencia2 integer, codigo_ocorrencia3 integer, codigo_ocorrencia4 integer, ocorrencia_classificacao varchar, ocorrencia_latitude numeric, ocorrencia_longitude numeric, ocorrencia_cidade varchar, ocorrencia_uf varchar, ocorrencia_pais varchar, ocorrencia_aerodromo varchar, ocorrencia_dia date, ocorrencia_hora time with time zone, investigacao_aeronave_liberada varchar, investigacao_status varchar, divulgacao_relatorio_numero varchar, divulgacao_relatorio_publicado varchar, divulgacao_dia_publicacao date, total_recomendacoes integer, total_aeronaves_envolvidas integer, ocorrencia_saida_pista varchar'
sql_querry = 'create table {} ({})'.format(nome_da_tabela, colunas)
cursor.execute(sql_querry)
conec.commit()
cursor.close()
conec.close()

In [190]:
#-----SALVANDO O DATA FRAME 'df_ocorrencia' DENTRO DA TABELA 'ocorrencia'-----#

columns = "'codigo_ocorrencia', 'codigo_ocorrencia1', 'codigo_ocorrencia2','codigo_ocorrencia3', 'codigo_ocorrencia4', 'ocorrencia_classificacao','ocorrencia_latitude', 'ocorrencia_longitude', 'ocorrencia_cidade','ocorrencia_uf', 'ocorrencia_pais', 'ocorrencia_aerodromo','ocorrencia_dia', 'ocorrencia_hora', 'investigacao_aeronave_liberada','investigacao_status', 'divulgacao_relatorio_numero','divulgacao_relatorio_publicado', 'divulgacao_dia_publicacao','total_recomendacoes', 'total_aeronaves_envolvidas','ocorrencia_saida_pista'"
df_ocorrencia.to_csv('ocorrencia', header=columns, index=False, encoding='utf-8')
arquivo_aberto_para_upload = open('ocorrencia', encoding='utf-8')

consulta_sql_para_upload = """
COPY ocorrencia FROM STDIN WITH
    CSV HEADER
    DELIMITER ','
    ENCODING 'utf-8'
"""

cursor.copy_expert(sql=consulta_sql_para_upload, file=arquivo_aberto_para_upload)
conec.commit()
cursor.close()
conec.close()

print('Arquivo adicionado no banco de dados {} com sucesso'.format(dbname))

Arquivo adicionado no banco de dados projeto_acidentes_aeroviarios com sucesso
